In [ ]:
#import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pyreadstat

In [ ]:
#load funds index/non-index data
crsp_data = pd.read_excel("Data/CRSP_fund.xlsx")
crsp_data

In [ ]:
print(crsp_data.columns)

In [ ]:
#subset data and rename columns for ease
crsp_data2 = crsp_data[['Date','Fund Identifier','CUSIP (8-digit)','Fund Name', 'index_fund_flag']]
crsp_cols = ['Date', 'fundid(CRSP)', 'fund_CUSIP', 'fundname(CRSP)', 'indexflag']
crsp_data2.columns = crsp_cols

In [ ]:
#Only select the index funds (B and D)
index_flag_funds = crsp_data2[crsp_data2['indexflag'].isin(['B', 'D'])]
index_flag_funds

In [ ]:
#load funds holdings data
lseg_data, lseg_meta = pyreadstat.read_dta("Data/LSEG_fund.dta")
lseg_data

In [ ]:
lseg_data.columns

In [ ]:
#subset data and rename columns for ease
lseg_data2 = lseg_data[lseg_data['shrout2'].isna() == False]
lseg_data3 = lseg_data2[['fdate','fundno', 'fundname','cusip','stkname', 'ticker','shrout2']]
lseg_cols = ['Date', 'fundid(LSEG)', 'fundname(LSEG)', 'stock_CUSIP', 'stock_name', 'ticker', 'shares_outstanding']
lseg_data3.columns = lseg_cols
lseg_data3

In [ ]:
#convert date columns to datetime format
index_flag_funds['Date'] = pd.to_datetime(index_flag_funds['Date'])
lseg_data3['Date'] = pd.to_datetime(lseg_data3['Date'])

In [ ]:
#filter data from 2021 onwards
lseg_data3 = lseg_data3[lseg_data3['Date'] >= "2021-01-01"]
lseg_data3

In [ ]:
#load WFICN to CRSP mapping data
CRSP_WFICN = pd.read_excel("Data/CRSP_WFICN.xlsx")
CRSP_WFICN2 = CRSP_WFICN[['Fund Identifier', 'Wharton Financial Instution Center Number']]
CRSP_WFICN2.columns = ['fundid(CRSP)', 'wficn']
CRSP_WFICN2

In [ ]:
#merge with index funds data
index_flag_funds2 = pd.merge(index_flag_funds, CRSP_WFICN2, how = "inner", on="fundid(CRSP)")
index_flag_funds2

In [ ]:
#load WFICN to LSEG mapping data
FUNDNO_WFICN = pd.read_stata("Data/FUNDO_WFICN.dta")
FUNDNO_WFICN2 = FUNDNO_WFICN[['fundno', 'wficn']]
FUNDNO_WFICN2.columns = ['fundid(LSEG)', 'wficn']
FUNDNO_WFICN2

In [ ]:
#remove duplicates and missing values
FUNDNO_WFICN3 = FUNDNO_WFICN2.drop_duplicates(subset=['wficn', 'fundid(LSEG)'], keep='first').reset_index(drop=True)
FUNDNO_WFICN3 = FUNDNO_WFICN3.dropna()
FUNDNO_WFICN3

In [ ]:
#merge with funds holdings data
lseg_data4 = pd.merge(lseg_data3, FUNDNO_WFICN3, how = "left", on = 'fundid(LSEG)')
lseg_data4 = lseg_data4.dropna()
lseg_data4

In [ ]:
#make funds dataset
merged_data = pd.merge(index_flag_funds2, lseg_data4, on=['Date', 'wficn'], how='inner')
merged_data

In [ ]:
#sum over to get each stock passively held
passive_data = (merged_data.groupby(['Date', 'ticker'])['shares_outstanding'].sum().reset_index())

In [ ]:
#subset data and rename columns for ease
passive_data.columns = ['Date', 'ticker', 'shares_held_passive']
passive_data_21 = passive_data.copy()
passive_data_21

In [ ]:
#write to csv so that don't need to load the big datasets again
passive_data_21.to_csv("Data/passive_21.csv")

In [ ]:
#Read data, new starting point
passive_data_21 = pd.read_csv("Data/passive_21.csv")

In [ ]:
#drop na
passive_data2 = passive_data_21.dropna()
passive_data3 = passive_data2[['Date', 'ticker', 'shares_held_passive']]
passive_data3

In [ ]:
#load earnings dates data
earnings_dates = pd.read_excel("Data/LSEG_earnings.xlsx")
earnings_dates

In [ ]:
#rename and subset columns for ease
earnings_dates2 = earnings_dates[['oftic', 'Period End Date, SAS Format' ,'Announce Date, SAS Format', 'Announce time, SAS Format']]
earnings_dates2.columns = ['ticker', 'quarter' ,'earnings_date', 'earnings_time']
earnings_dates2

In [ ]:
#convert to datetime format
earnings_dates2['earnings_date'] = pd.to_datetime(earnings_dates2['earnings_date'])
earnings_dates2['earnings_time'] = pd.to_datetime(earnings_dates2['earnings_time'], format='%H:%M:%S').dt.time
earnings_dates2

In [ ]:
#if earnings time is after mkt close consider it next day
earnings_dates2['nextday'] = earnings_dates2['earnings_time'] > pd.to_datetime('16:30:00', format='%H:%M:%S').time()


In [ ]:
#change earnings date to next day if after mkt close
if earnings_dates2['nextday'].any() == True:
    earnings_dates2['earnings_date'] = earnings_dates2['earnings_date'] + pd.Timedelta(days=1)

earnings_dates2

In [ ]:
#subset and add additional Date column for merging
earnings_dates3 = earnings_dates2.drop(columns=['earnings_time', 'nextday', 'quarter'])
earnings_dates3['Date'] = earnings_dates3['earnings_date']
earnings_dates3

In [ ]:
#load stock data
stock_data = pd.read_stata("Data/daily_stock.dta")

In [ ]:
#subset to relevant time and convert date to datetime format
stock_data = stock_data[stock_data['date'] >= "2021-01-01"]
stock_data['date'] = pd.to_datetime(stock_data['date'])

In [ ]:
#rename columns and create shares outstanding column
stock_colnames = ["permno", "Date", "ticker", "company", "cusip", "price", "vol", "return", "shares_outstnading(1000s)"]
stock_data.columns = stock_colnames
stock_data["shares_outstanding"] = stock_data["shares_outstnading(1000s)"] * 1000 

In [ ]:
#subset and drop na
stock_data2 = stock_data.drop("shares_outstnading(1000s)", axis = 1)
stock_data3 = stock_data2.dropna()
stock_data3

In [ ]:
#find negative prices
negative_prices = stock_data3[(stock_data3.duplicated(subset=['ticker','Date'], keep=False)) &(stock_data3['price'] < 0)]
negative_prices

In [ ]:
#remove negative prices
stock_data3 = stock_data3[stock_data3['price'] >= 0]

In [ ]:
#drop duplicates keeping the one with highest shares outstanding, after check those seem more reasonable
stock_data3 = stock_data3.sort_values(['ticker','Date','shares_outstanding'], ascending=[True, True, False])
stock_data3 = stock_data3.drop_duplicates(subset=['ticker','Date'], keep='first')

In [ ]:
#clean data for merge
passive_data3['ticker'] = passive_data3['ticker'].astype(str)
passive_data3['ticker'] = passive_data3['ticker'].str.upper()
stock_data3['ticker'] = stock_data3['ticker'].astype(str)
stock_data3['ticker'] = stock_data3['ticker'].str.upper()
passive_data3['Date'] = pd.to_datetime(passive_data3['Date'])
stock_data3['Date'] = pd.to_datetime(stock_data3['Date'])
passive_data3 = passive_data3.sort_values(['ticker', 'Date']).reset_index(drop=True)
stock_data3 = stock_data3.sort_values(['ticker', 'Date']).reset_index(drop=True)

In [ ]:
#merge datasets
merge1 = pd.merge(stock_data3, passive_data3, on = ['ticker', 'Date'], how = 'left')
merge1

In [ ]:
#backfill passive holdings for each stock for each quarter
merge2 = merge1.copy()
merge2['QuarterEnd'] = merge2['Date'].dt.to_period('Q').dt.end_time
merge2['shares_held_passive'] = (merge2.groupby(['ticker', 'QuarterEnd'])['shares_held_passive']
    .transform(lambda x: x.bfill() if x.notna().any() else x))
merge2

In [ ]:
#drop na and create share passive column
merge3 = merge2.dropna()
merge3 = merge3.drop(columns=['QuarterEnd'])
merge3['share_passive'] = merge3['shares_held_passive']/merge3['shares_outstanding']
merge3

In [ ]:
#prep data for merge
earnings_dates3['ticker'] = earnings_dates3['ticker'].astype(str)
earnings_dates3['ticker'] = earnings_dates3['ticker'].str.upper()
earnings_dates3['Date'] = pd.to_datetime(earnings_dates3['Date'])
earnings_dates3 = earnings_dates3.sort_values(['ticker', 'Date']).reset_index(drop=True)

In [ ]:
#merge
merge4 = pd.merge(merge3, earnings_dates3, on = ['Date', 'ticker'], how = 'left')
merge4

In [ ]:
#identify stocks with no earnings dates
no_earn_tickers = merge4.groupby('ticker')['earnings_date'].apply(lambda x: x.notna().any() == False)
no_earn_tickers = no_earn_tickers[no_earn_tickers].index.tolist()
len(no_earn_tickers)

In [ ]:
#remove stocks with no earnings dates
merge4 = merge4[~merge4['ticker'].isin(no_earn_tickers)].copy()
merge4

In [ ]:
#new column for next earnings date
merge4['earnings_next'] = (merge4.groupby('ticker')['earnings_date'].transform(lambda x: x.bfill()))
merge4

In [ ]:
#new column for previous earnings date
merge4['earnings_prev'] = (merge4.groupby('ticker')['earnings_date'].transform(lambda x: x.ffill()))
merge4

In [ ]:
#days since previous and until next earnings date
merge4['days_since_prev'] = (merge4['Date'] - merge4['earnings_prev']).dt.days
merge4['days_until_next'] = (merge4['earnings_next'] - merge4['Date']).dt.days
merge4

In [ ]:
#function to choose nearest earnings date
def choose_nearest(row):
    # If both exist, pick the closer date in absolute time
    if pd.notna(row['earnings_prev']) and pd.notna(row['earnings_next']):
        return row['earnings_prev'] if abs(row['days_since_prev']) <= abs(row['days_until_next']) else row['earnings_next']
    # If only one exists, use it
    elif pd.notna(row['earnings_prev']):
        return row['earnings_prev']
    elif pd.notna(row['earnings_next']):
        return row['earnings_next']
    else:
        return np.nan

In [ ]:
#apply the function
merge4['earnings_date_nearest'] = merge4.apply(choose_nearest, axis=1)
merge4

In [ ]:
#create day to/from nearest earnings date
merge4['days_from_earn'] = (merge4['Date'] - merge4['earnings_date_nearest']).dt.days
merge4

In [ ]:
#subset data
merge5 = merge4[['Date', 'ticker', 'price', 'vol', 'return', 'share_passive', 'days_from_earn']]
merge5

In [ ]:
merge5['return_lag1'] = merge5.groupby('ticker')['return'].shift(1)
merge5['abnormal_return'] = abs(merge5['return'] - merge5['return_lag1'])
merge5['sd_3day'] = merge5.groupby('ticker')['price'].rolling(window=3).std().reset_index(level=0, drop=True)
merge5['3day_vol'] = merge5['sd_3day']/np.sqrt(3)
merge5

In [ ]:
#create event window
master_data = merge5[(merge5['days_from_earn'] >= -31) & (merge5['days_from_earn'] <= 15)]
master_data

In [ ]:
#check if any share passive greater than 1
greaterthan1 = master_data[master_data['share_passive'] >= 1]
problem_tickers = greaterthan1['ticker'].unique()
totaltickers = master_data['ticker'].unique()
print(f"total:{len(totaltickers)}, problems: {len(problem_tickers)}")

In [ ]:
#remove those with share passive greater than 1
master_data = master_data[master_data['share_passive'] < 0.95] #maybe 1.0 or 0.9?
master_data

In [ ]:
master_data.to_csv("Data/master_data.csv")